In [ ]:
# Don't think you need it as long as this notebook is in the same folder as peerselect. 
import sys
sys.path.insert(0, "\peerselect")

In [ ]:
# Makes it easier to upodate changes without restarting the notebook. Ignore.
%load_ext autoreload
%autoreload 2

In [ ]:
%aimport

In [ ]:
### Standard Magic and startup initializers.

import math
import csv
import numpy as np
import random
import itertools
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from collections import Counter

from peerselect import impartial
from peerselect import profile_generator
from peerselect.estimate_eps import estimate_eps

%matplotlib inline
matplotlib.style.use('seaborn-whitegrid')

matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['font.size'] = 25
np.set_printoptions(precision=3)
np.set_printoptions(threshold=10)

In [ ]:
class Impartial:
    VANILLA = "Vanilla"
    EXACT = "ExactDollarPartition"
    PARTITION = "Partition"
    DPR = "DollarPartitionRaffle"
    CREDIABLE = "CredibleSubset"
    RAFFLE = "DollarRaffle"
    NOMINATION = "PeerNomination"
    ALL = (VANILLA, EXACT, PARTITION, RAFFLE, CREDIABLE, DPR, NOMINATION)

## Running the algorithms

### Generate an instance of the peer review  problem

In [ ]:
# Parameters
n = 120
k = 20
m = 7
l = 4
p = 0.8

# The agents are represented by integers that conveniently represent the ground truth
agents = np.arange(0, n)
# Generate individual rankings of agents (the voting profile) using Mallows model. 
# It is possible to mix different ground truths and different dispersion parameters
profile = profile_generator.generate_mallows_mixture_profile(agents, agents, [p, 1-p], [agents, agents], [0.5, 1.0])
# Generate the clustering into l clusters for the EDP assignment.
clustering = impartial.even_partition_order(sorted(agents, key=lambda j: random.random()), l)

# We need some ways to convert rankings into scores for EDP:
# - Borda scores -- need to start at 1 to distinguish from non-review in the score matrix
scores = np.arange(m, 0, -1)
# - Approval scores a-la PeerNomination
nom_quota = (k/n)*m
scores_pn = np.concatenate((np.ones(math.floor(nom_quota)),
        [nom_quota-math.floor(nom_quota)],
        np.zeros(m-math.ceil(nom_quota))))

# Generate an m-regular assignment
m_assignment = profile_generator.generate_approx_m_regular_assignment(agents, m, clustering, randomize=False)

# Combine the assignment and profile into a score matrix, refer to code for format. 
score_matrix = profile_generator.strict_m_score_matrix(profile, m_assignment, scores)

### Now we can run the algorithms
The output of each algorithm is simply the list of accepted (winning) agents

Some examples:

In [ ]:
# EDP
impartial.exact_dollar_partition_explicit(score_matrix, k, clustering, normalize=True)

In [ ]:
# We need to use the slack parameter for PeerNomination. Use this function to get the best estimate.
eps_estimate = estimate_eps(n, m, k)

# Run PeerNomination without weights
impartial.peer_nomination_lottery(score_matrix, k, eps_estimate)

In [ ]:
# PeerNomination with Distance. It usually returns too many agents so can set slack parameter to 0 as a heuristic. 
impartial.weighted_peer_nomination(score_matrix, k, impartial.dist_weights, 0)

 ## Testing the Algorithms
 Here's a simple way to run some experiments and collect data

In [ ]:
_DEBUG = False

#random.seed(15)

# Warning! It's quite slow for a big number of parameters or iterations. 
# For the paralellised version, use scripts in the experiment folder.

# Number of iterations for each combination of parameters
s = 10
# Range of parameters for testing
test_n = [120]
test_k = [20, 25, 30]
test_m = [7, 8, 9]
test_l = [4]
test_p = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.9, 1]

# Map for all Results.
gt_results = {}
pn_sizes = {}  #track output size of peer nomination
for n,k,m,l,p in itertools.product(test_n, test_k, test_m, test_l, test_p):
    agents = np.arange(0, n)
    
    eps_estimate = estimate_eps(n, m, k)
    
    for c_sample in range(s):
        # Generate a profile and clustering
        profile = profile_generator.generate_mallows_mixture_profile(agents, agents, [p, 1-p], [agents, agents], [0.5, 1])
        clustering = impartial.even_partition_order(sorted(agents, key=lambda j: random.random()), l)
        
        # Borda -- need to start at 1 to distinguish from non-review in the score matrix
        scores = np.arange(m, 0, -1)
        
        # Approval a-la PeerNomination
        nom_quota = (k/n)*m
        scores_pn = np.concatenate((np.ones(math.floor(nom_quota)),
                [nom_quota-math.floor(nom_quota)],
                np.zeros(m-math.ceil(nom_quota))))
        
        # Generate an m-regular assignment
        m_assignment = profile_generator.generate_approx_m_regular_assignment(agents, m, clustering, randomize=False)
        
        score_matrix = profile_generator.strict_m_score_matrix(profile, m_assignment, scores)
        score_matrix_pn = profile_generator.strict_m_score_matrix(profile, m_assignment, scores_pn)
        
        # Capture the winning sets
        ws = {}

        # Run peer nomination using the estimated epsilon
        ws[Impartial.NOMINATION] = impartial.peer_nomination_lottery(score_matrix, k, eps_estimate)
        ws[Impartial.EXACT] = impartial.exact_dollar_partition_explicit(score_matrix, k, clustering, normalize=True)
        ws["EDP_rank"] = impartial.exact_dollar_partition_explicit(score_matrix_pn, k, clustering, normalize=True)
        ws["PN_dist"] = impartial.weighted_peer_nomination(score_matrix, k, impartial.dist_weights, eps_estimate)
        ws["PN_maj"] = impartial.weighted_peer_nomination(score_matrix, k, impartial.maj_weights, eps_estimate)
        ws["PN_step"] = impartial.weighted_peer_nomination(score_matrix, k, impartial.step_weights, eps_estimate)

        for x in [Impartial.NOMINATION, Impartial.EXACT, "EDP_rank", "PN_dist", "PN_maj", "PN_step"]:
            key = (n, k, m, l, p, s, x)
            gt_results[key] = gt_results.get(key, []) + [len(set(np.arange(0, k)) & set(ws[x]))]
            pn_sizes[key] = pn_sizes.get(key, []) + [len(set(ws[x]))]

        key = (n, k, m, l, p)
        
    print("Finished: " + ",".join([str(x) for x in [n, k, m, l, p, s]]))
    
print("All done!")

df = pd.DataFrame(gt_results)
df.columns.names = ['n', 'k', 'm', 'l', 'p', 's', 'algo']

df_sizes = pd.DataFrame(pn_sizes)
df_sizes.columns.names = ['n', 'k', 'm', 'l', 'p', 's', 'algo']


Saving and loading the results

In [ ]:
df.to_pickle("peerselect/db/some_exp.pkl") # saving the results

In [ ]:
df = pd.read_pickle("peerselect/db/some_exp.pkl") # loading the results

Filter the data for plotting

In [ ]:
xlabels = [0.5, 1]

df_filtered = df[
    [x for x in df.columns if x[4] in xlabels]
]
df_sizes = df_sizes[
    [x for x in df_sizes.columns if x[4] in xlabels]
]

In [ ]:
df_filtered.columns

Plotting the results

In [ ]:
#
# Takes a slice of the array and generates the graphs etc.
#
def make_output(df, df_sizes, test, labels, save_path=None):
    means = df.loc[:, test].mean().unstack()
    errors = df.loc[:, test].std().unstack()
    mins = df.loc[:, test].min().unstack()

#     algo_list = [Impartial.NOMINATION, "PN_dist", "PN_step", "PN_maj", Impartial.EXACT]
    algo_list = np.unique([x[-1] for x in df.columns])

    means = means[algo_list]
    errors = errors[algo_list]
    
    for index, row in means.iterrows():
        means.loc[index] = row / float(index[1])
    for index, row in errors.iterrows():
        errors.loc[index] = row / float(index[1])
    
    #Set colors..
#     color_list = plt.cm.Paired(np.linspace(0, 1, 10))
#     color_list = color_list[:7]
    color_list = ["#7fc97f","#beaed4","#fdc086","#ffff99","#386cb0"]
    color_list = color_list[1:5]
    #color_list = sns.color_palette("pastel", 6)
    
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    
    
    means.plot(kind='bar', ax=ax1, legend=False, yerr=errors.values.T, figsize=(20, 10),
            color=color_list, error_kw={'ecolor':'Black', 'linewidth':2, 'capsize':4}, width=0.85, edgecolor="k")
    ax1.set_ylabel("Recall")    
    ax1.set_ylim([0, 1])
    ax1.set_yticks(np.linspace(0, 1, 5))
    
#     print(means)
    
#     if df_sizes != None:
    means = df_sizes.loc[:, test].mean().unstack()
    errors = df_sizes.loc[:, test].std().unstack()
    mins = df_sizes.loc[:, test].min().unstack()

#     algo_list = [Impartial.NOMINATION, "PN_maj","PN_dist",  "PN_step", Impartial.EXACT]

    means = means[algo_list]
    errors = errors[algo_list]

    for index, row in means.iterrows():
        means.loc[index] = row / float(index[1])
    for index, row in errors.iterrows():
        errors.loc[index] = row / float(index[1])

    means.plot(kind='bar', ax=ax2, legend=False, yerr=errors.values.T, figsize=(20, 10),
            color=color_list, error_kw={'ecolor':'Black', 'linewidth':2, 'capsize':4}, width=0.85, edgecolor="k")
#     plt.title(f"Adversarial Fair Test, n={test[0]}, k={test[1]}, m={test[2]}, l={test[3]}")
#     plt.legend(["PN+Unit", "PN+Dist", "PN+Step", "PN+Maj", "EDP"], bbox_to_anchor = (0,-0.05,1,1), bbox_transform=plt.gcf().transFigure, loc='lower center', ncol=7, borderaxespad=0.)

    ax2.set_ylabel("Relative Size")
    ax2.set_xlabel("Proportion of accurate reviewers")
#     ax2.axhline(1, c="0.5", ls="--")
    
    print()
    plt.legend(algo_list, bbox_to_anchor = (0,-0.05,1,1), bbox_transform=plt.gcf().transFigure, loc='lower center', ncol=7, borderaxespad=0.)
    plt.gca().set_xticklabels(labels)
    plt.xticks(rotation = 0)
    plt.yticks(np.linspace(0, 1, 5))

    if save_path != None:
        plt.savefig(save_path, bbox_inches="tight", dpi=400)    
    else:
        plt.show()
#     print(means)
    return means

make_output(df, df_sizes, (120, 25, 8, 4, slice(None), 1), xlabels)
# Tbh I don't really know how this works, I just modified it enough to show the plots I want.